In [ ]:
import uproot
import numpy as np

file_prompt = uproot.open("Prompt_DstarToD0Pi.root")
tree_prompt = file_prompt["treeMLDstar"]

print(file_prompt.keys())
tree_prompt.keys()


['treeMLDstar;1']


['fCandidateSelFlag',
 'fChi2PCAD0',
 'fCosThetaStarD0',
 'fCpaD0',
 'fCpaXYD0',
 'fDecayLengthD0',
 'fDecayLengthNormalisedD0',
 'fDecayLengthXYD0',
 'fDecayLengthXYNormalisedD0',
 'fEta',
 'fEtaD0',
 'fFlagMcMatchRec',
 'fImpParamSoftPi',
 'fImpactParameter0',
 'fImpactParameter1',
 'fImpactParameterNormalised0',
 'fImpactParameterNormalised1',
 'fImpactParameterNormalisedSoftPi',
 'fImpactParameterProductD0',
 'fM',
 'fMD0',
 'fMaxNormalisedDeltaIPD0',
 'fNSigTofKa0',
 'fNSigTofKa1',
 'fNSigTofKaSoftPi',
 'fNSigTofPi0',
 'fNSigTofPi1',
 'fNSigTofPiSoftPi',
 'fNSigTpcKa0',
 'fNSigTpcKa1',
 'fNSigTpcKaSoftPi',
 'fNSigTpcPi0',
 'fNSigTpcPi1',
 'fNSigTpcPiSoftPi',
 'fNSigTpcTofKa0',
 'fNSigTpcTofKa1',
 'fNSigTpcTofKaSoftPi',
 'fNSigTpcTofPi0',
 'fNSigTpcTofPi1',
 'fNSigTpcTofPiSoftPi',
 'fPhi',
 'fPhiD0',
 'fPt',
 'fPtBhadMother',
 'fPtD0',
 'fPtProng0',
 'fPtProng1',
 'fPtSoftPi',
 'fY',
 'fYD0',
 'fInvDeltaMass']

In [ ]:
file_prompt = uproot.open("Prompt_DstarToD0Pi.root")
file_nonprompt = uproot.open("Nonprompt_DstarToD0Pi.root")
file_bkg = uproot.open("Bkg_DstarToD0Pi.root")

tree_prompt = file_prompt["treeMLDstar"]
tree_nonprompt = file_nonprompt["treeMLDstar"]
tree_bkg = file_bkg["treeMLDstar"]

# -----------------------------
# Branches to load
# -----------------------------

branches = [
    # Topological variables
    "fCpaD0",
    "fCpaXYD0",
    "fDecayLengthXYD0",
    "fImpactParameterProductD0",
    "fImpParamSoftPi",
    "fMaxNormalisedDeltaIPD0",

    # PID prong0
    "fNSigTpcPi0",
    "fNSigTpcKa0",
    "fNSigTofPi0",
    "fNSigTofKa0",

    # PID prong1
    "fNSigTpcPi1",
    "fNSigTpcKa1",
    "fNSigTofPi1",
    "fNSigTofKa1"
]

# -----------------------------
# Load arrays
# -----------------------------

arr_prompt = tree_prompt.arrays(branches, library="np")
arr_nonprompt = tree_nonprompt.arrays(branches, library="np")
arr_bkg = tree_bkg.arrays(branches, library="np")

# -----------------------------
# Build feature matrices
# -----------------------------

def build_X(arr):
    return np.column_stack([arr[b] for b in branches])

X_prompt = build_X(arr_prompt)
X_nonprompt = build_X(arr_nonprompt)
X_bkg = build_X(arr_bkg)

# -----------------------------
# Create labels
# -----------------------------

y_prompt = np.zeros(len(X_prompt))        # 0
y_nonprompt = np.ones(len(X_nonprompt))   # 1
y_bkg = 2 * np.ones(len(X_bkg))            # 2

# -----------------------------
# Merge datasets
# -----------------------------

X = np.vstack((X_prompt, X_nonprompt, X_bkg))
y = np.concatenate((y_prompt, y_nonprompt, y_bkg))

# -----------------------------
# Sanity checks
# -----------------------------

print("X shape:", X.shape)
print("y shape:", y.shape)
print("First 10 labels:", y[:10])

X shape: (12949176, 14)
y shape: (12949176,)
First 10 labels: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
X = np.nan_to_num(X, nan=0.0)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# -----------------------------
# Train-test split (70/30)
# -----------------------------

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.30,
    random_state=42,
    shuffle=True
)

# -----------------------------
# Feature normalization
# -----------------------------

scaler = StandardScaler()

# Fit ONLY on training data
X_train_scaled = scaler.fit_transform(X_train)

# Apply same transformation to test data
X_test_scaled = scaler.transform(X_test)

# -----------------------------
# Sanity check
# -----------------------------

print("Mean of first feature (train):", X_train_scaled[:,0].mean())
print("Std of first feature (train):", X_train_scaled[:,0].std())


Mean of first feature (train): -9.1639365e-08
Std of first feature (train): 1.0


In [ ]:
import torch
import torch.nn as nn

class D0Classifier(nn.Module):
    def __init__(self):
        super(D0Classifier, self).__init__()
        
        self.fc1 = nn.Linear(14, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 3)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)   # logits
        return x


ModuleNotFoundError: No module named 'torch'

In [ ]:

# Create model
model = D0Classifier()

# Loss function
criterion = torch.nn.CrossEntropyLoss()

# Optimizer (Adam)
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=0.001
)

# Learning-rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=20,   # reduce LR every 20 epochs
    gamma=0.5       # LR_new = LR_old * 0.5
)


In [ ]:
print("NaNs in X:", np.isnan(X).sum())
print("Infs in X:", np.isinf(X).sum())


NaNs in X: 0
Infs in X: 0


In [ ]:

# Convert numpy arrays to torch tensors
X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)

X_test_t = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.long)

# Number of epochs
n_epochs = 60

train_losses = []

# Training loop
for epoch in range(n_epochs):

    # Set model to training mode
    model.train()

    # Forward pass
    outputs = model(X_train_t)

    # Compute loss
    loss = criterion(outputs, y_train_t)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Scheduler step
    scheduler.step()

    # Store loss
    train_losses.append(loss.item())

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{n_epochs}, Loss = {loss.item():.4f}")


KeyboardInterrupt: 

In [ ]:
print(torch.version.cuda)
print(torch.cuda.is_available())

NameError: name 'torch' is not defined

In [ ]:

from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc


# -----------------------------
# Set model to evaluation mode
# -----------------------------
model.eval()

with torch.no_grad():
    outputs_test = model(X_test_t)
    probs_test = torch.softmax(outputs_test, dim=1)
    preds_test = torch.argmax(probs_test, dim=1).numpy()

# -----------------------------
# Accuracy
# -----------------------------
acc = accuracy_score(y_test, preds_test)
print("Test Accuracy:", acc)

# -----------------------------
# Confusion Matrix
# -----------------------------
cm = confusion_matrix(y_test, preds_test)
print("Confusion Matrix:\n", cm)

# -----------------------------
# ROC: Prompt vs Rest
# -----------------------------
y_true_prompt = (y_test == 0).astype(int)
scores_prompt = probs_test[:,0].numpy()

fpr_p, tpr_p, _ = roc_curve(y_true_prompt, scores_prompt)
auc_p = auc(fpr_p, tpr_p)

plt.figure()
plt.plot(fpr_p, tpr_p, label=f"Prompt vs Rest (AUC = {auc_p:.3f})")
plt.plot([0,1],[0,1],'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.grid()
plt.show()

# -----------------------------
# ROC: Nonprompt vs Rest
# -----------------------------
y_true_np = (y_test == 1).astype(int)
scores_np = probs_test[:,1].numpy()

fpr_np, tpr_np, _ = roc_curve(y_true_np, scores_np)
auc_np = auc(fpr_np, tpr_np)

plt.figure()
plt.plot(fpr_np, tpr_np, label=f"Nonprompt vs Rest (AUC = {auc_np:.3f})")
plt.plot([0,1],[0,1],'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.grid()
plt.show()
